# Predicción del resultado de partidas de ajedrez a partir de su metadata.

**Palabras clave:**

- **ELO:** valor numérico que determina la fuerza de un jugador de ajedrez. En Costa Rica, se inicia con 1400 puntos y se alcanza la Primera División con un ELO de 2000, mientras que el campeón nacional tiene una fuerza cercana a 2400. El campeón del mundo, actualmente, posee un ELO de 2855. 
- **Lichess:** sitio web *open-source* muy popular para jugar ajedrez en línea, con millones de partidas jugadas cada mes. En Internet, los ELOs tienden a ser más altos (debido a las mecánicas que utilizan las páginas para calcularlo).
- **Apertura:** primeras jugadas que se realizan en una partida de ajedrez. Típicamente, corresponde a las primeras 10 o 15 movidas.
- **ECO:** *Encyclopedia of Chess Openings*; clasificación alfanumérica para todas las aperturas posibles en el ajedrez. Cada código consta de una letra (de la A a la E) seguida de dos dígitos.
- **Control de tiempo:** minutos que posee cada jugador para la partida. Una partida *bullet* significa 1 minuto por jugador; una partida *blitz* consta de 5 minutos por jugador.
- **Tablas:** empate en ajedrez.

## 1. Descripción del problema y justificación del enfoque a utilizar

Con base en las características de una partida de ajedrez (ELO de sus jugadores, apertura jugada, control de tiempo empleado, cantidad de jugadas, entre otros) se desea predecir quién ganó el juego. Para esto, se utilizarán técnicas de aprendizaje automático para clasificación binaria, donde "1" significa victoria de las Blancas, mientras que "0" significa victoria de las Negras. En este trabajo, no se tomarán en cuenta las partidas que terminaron en tablas, por lo que una clasificación binaria se ajusta bien al escenario planteado.

### Dataset a utilizar

Se utilizará un *dataset* obtenido de la página [Lichess.org](https://lichess.org/), con cerca de 20 000 partidas jugadas en septiembre del 2017. El *dataset* fue convertido de PGN a CSV por un usuario de Kaggle y se encuentra disponible en dicha página.

Puede encontrar el conjunto de datos a utilizar haciendo click en este [enlace](https://www.kaggle.com/datasnaek/chess).